In [645]:
# Importing libraries for data preprocessing
import pandas as pd
import numpy as np
from sklearn import preprocessing
from collections import Counter
from sklearn.preprocessing import StandardScaler
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder 

In [646]:
# importing keras libraries for model building
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [647]:
# Loading the data
data=pd.read_csv('TRAINING.csv')

In [648]:
data.set_index('id',inplace=True)

In [649]:
data.head()

,Area(total),Troom,Nbedrooms,Nbwashrooms,Twashrooms,roof,Roof(Area),Lawn(Area),Nfloors,API,ANB,Grade,Price
id,,,,,,,,,,,,,
1,305,8.0,2.0,1.0,3.0,NO,0.0,76.0,2,63.0,4,D,3826$
2,344,6.0,7.0,7.0,8.0,no,0.0,83.0,7,97.0,1,B,4458$
3,294,8.0,4.0,3.0,4.0,yes,97.0,78.0,1,88.0,4,C,4199$
4,328,5.0,4.0,2.0,4.0,NO,0.0,91.0,5,86.0,1,D,3462$
5,295,9.0,5.0,3.0,5.0,NaN,NaN,97.0,4,93.0,3,C,4543$


In [650]:
tgg=data['Grade'].tolist()

In [651]:
type(tgg[0])

str

In [652]:
grade_classes=data['Grade'].nunique()
print(grade_classes)

5


In [653]:
data['Grade'].nunique()

5

In [654]:
numerical_col=['Area(total)','Roof(Area)','Lawn(Area)','Price']
categorical_col=['Troom','Nbedrooms','Nbwashrooms','Twashrooms','roof','Nfloors','API']

In [655]:
len(data)

7000

In [656]:
data.isna().any()

Area(total)    False
Troom           True
Nbedrooms       True
Nbwashrooms     True
Twashrooms      True
roof            True
Roof(Area)      True
Lawn(Area)      True
Nfloors        False
API             True
ANB            False
Grade          False
Price          False
dtype: bool

In [657]:
# making all the string value to lower of column
def text_lower(df,col):
    df[col]=df[col].str.lower()

In [658]:
def clean_price(df,col):
    s = df[col].str.replace('$','')
    df[col]=pd.to_numeric(s)

In [659]:
def to_numeric(df,col):
    df[col]=pd.to_numeric(df[col])

In [660]:
# function to make dtype of column as category
def category_type(df,col):
    df[col] = df[col].astype('category')

In [661]:
text_lower(data,'roof')
clean_price(data,'Price')

In [662]:
# funtion to replace the null values with the most common for categorical variable
def fill_categorical_na(df,col):
    most_cmn=Counter(df[col]).most_common(2)[1][0]
    print(most_cmn)
    df[col].fillna(most_cmn,inplace=True)

In [663]:
# funtion to replace the null values with the mean for numerical variable
def fill_numerical_na(df,col):
    mn=df[col].mean()
    #print(col)
    print(col,mn)
    df[col].fillna(value=mn,inplace=True)

In [664]:
for i in numerical_col:
    to_numeric(data,i)
# for j in categorical_col:
#     category_type(j)
for i in categorical_col:
    fill_categorical_na(data,i)
for j in numerical_col:
    fill_numerical_na(data,j)

5.0
4.0
5.0
7.0
yes
4
78.0
Area(total) 325.2255714285714
Roof(Area) 47.20276252877634
Lawn(Area) 89.95798799657045
Price 4036.4537142857143


In [665]:
# encoding the categorical variables
from sklearn.preprocessing import LabelBinarizer
dat=[]
def label_enco(df,col):
    values=np.array(df[col].tolist())
    label_encoder = preprocessing.LabelEncoder() 
    #label_encoder.fit(data[col])
    int_enco=label_encoder.fit_transform(values)
    
    #int_enco=int_enco.reshape(len(int_enco),1)
    
    #     lb=LabelBinarizer()
    #     lb_result=lb.fit_transform(int_enco.reshape(-1,1))
    #     print(type(lb_result))
    #     #print(lb_result)
    #     dt=pd.DataFrame(lb_result,columns=lb.classes_)
    #     #dat= pd.concat([dat, dt], axis=1)
    #     dat.append(dt)
    
    
    onehotencoder = OneHotEncoder(sparse=False) 
    int_enco=int_enco.reshape(len(int_enco),1)
    onehot_encoded=onehotencoder.fit_transform(int_enco)
    df4=pd.DataFrame(onehot_encoded).add_prefix(col+'_')
    dat.append(df4)
    #pd.concat([data, df4], axis=1,)
    
    
    #     print(onehot_encoded)
    #     print("Kahoooooo")
    #     print(len(onehot_encoded))
    #     print(type(onehot_encoded))
    #     print(onehot_encoded[1])
    #     print("dusra")
    #     temp=[]
    #     for i in onehot_encoded:
    #         temp.append(i)
    #     data[col]=temp 
    
    return(label_encoder,onehotencoder)
    

In [666]:
category_type(data,'Grade')
values=np.array(data['Grade'].tolist())
grade_encoder = preprocessing.LabelEncoder() 
#label_encoder.fit(data[col])
grade_enco=grade_encoder.fit_transform(values)
data['Grade']=grade_enco

In [667]:

def one_hot_enco(col):
    onehotencoder = OneHotEncoder(categories='auto',sparse=False) 
    onehotencoder.fit(data[col].values.reshape(-1,1))
    data[col]=onehotencoder.transform(data[col].values.reshape(-1,1)).toarray()
    return(onehotencoder)

In [668]:
def get_dummy(col):
    data[col]=pd.get_dummies(data[col].values)

In [669]:
# scalling the dataset for better performance of model and reducing complexity
from sklearn.preprocessing import StandardScaler
def numeric_scaller(df,col):
    scaller=StandardScaler()
    df[col]=scaller.fit_transform(df[col].values.reshape(-1,1))
    return(scaller)

In [670]:
label_enco_ins=[]
one_hot_ins=[]
for i in categorical_col:
    s,p=label_enco(data,i)
    label_enco_ins.append(s)
    one_hot_ins.append(p)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "

In [671]:
dat

[      Troom_0  Troom_1  Troom_2  Troom_3  Troom_4
 0         0.0      0.0      0.0      1.0      0.0
 1         0.0      1.0      0.0      0.0      0.0
 2         0.0      0.0      0.0      1.0      0.0
 3         1.0      0.0      0.0      0.0      0.0
 4         0.0      0.0      0.0      0.0      1.0
 ...       ...      ...      ...      ...      ...
 6995      0.0      0.0      0.0      1.0      0.0
 6996      1.0      0.0      0.0      0.0      0.0
 6997      0.0      1.0      0.0      0.0      0.0
 6998      0.0      0.0      0.0      1.0      0.0
 6999      1.0      0.0      0.0      0.0      0.0
 
 [7000 rows x 5 columns],
       Nbedrooms_0  Nbedrooms_1  Nbedrooms_2  Nbedrooms_3  Nbedrooms_4  \
 0             1.0          0.0          0.0          0.0          0.0   
 1             0.0          0.0          0.0          0.0          0.0   
 2             0.0          0.0          1.0          0.0          0.0   
 3             0.0          0.0          1.0          0.0       

In [672]:
data.head()

,Area(total),Troom,Nbedrooms,Nbwashrooms,Twashrooms,roof,Roof(Area),Lawn(Area),Nfloors,API,ANB,Grade,Price
id,,,,,,,,,,,,,
1,305,8.0,2.0,1.0,3.0,no,0.000000,76.0,2,63.0,4,3,3826
2,344,6.0,7.0,7.0,8.0,no,0.000000,83.0,7,97.0,1,1,4458
3,294,8.0,4.0,3.0,4.0,yes,97.000000,78.0,1,88.0,4,2,4199
4,328,5.0,4.0,2.0,4.0,no,0.000000,91.0,5,86.0,1,3,3462
5,295,9.0,5.0,3.0,5.0,yes,47.202763,97.0,4,93.0,3,2,4543


In [673]:
type(data['Troom'].tolist())

list

In [674]:
num_scaller_ins=[]
for i in numerical_col:
    c=numeric_scaller(data,i)
    num_scaller_ins.append(c)

In [675]:
num_scaller_ins

[StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True),
 StandardScaler(copy=True, with_mean=True, with_std=True)]

In [676]:
data.head()

,Area(total),Troom,Nbedrooms,Nbwashrooms,Twashrooms,roof,Roof(Area),Lawn(Area),Nfloors,API,ANB,Grade,Price
id,,,,,,,,,,,,,
1,-0.984255,8.0,2.0,1.0,3.0,no,-1.076883,-1.181860,2,63.0,4,3,-0.423898
2,0.913637,6.0,7.0,7.0,8.0,no,-1.076883,-0.589151,7,97.0,1,1,0.849083
3,-1.519558,8.0,4.0,3.0,4.0,yes,1.136073,-1.012515,1,88.0,4,2,0.327403
4,0.135015,5.0,4.0,2.0,4.0,no,-1.076883,0.088230,5,86.0,1,3,-1.157071
5,-1.470894,9.0,5.0,3.0,5.0,yes,0.000000,0.596266,4,93.0,3,2,1.020291


In [677]:
data.head()

,Area(total),Troom,Nbedrooms,Nbwashrooms,Twashrooms,roof,Roof(Area),Lawn(Area),Nfloors,API,ANB,Grade,Price
id,,,,,,,,,,,,,
1,-0.984255,8.0,2.0,1.0,3.0,no,-1.076883,-1.181860,2,63.0,4,3,-0.423898
2,0.913637,6.0,7.0,7.0,8.0,no,-1.076883,-0.589151,7,97.0,1,1,0.849083
3,-1.519558,8.0,4.0,3.0,4.0,yes,1.136073,-1.012515,1,88.0,4,2,0.327403
4,0.135015,5.0,4.0,2.0,4.0,no,-1.076883,0.088230,5,86.0,1,3,-1.157071
5,-1.470894,9.0,5.0,3.0,5.0,yes,0.000000,0.596266,4,93.0,3,2,1.020291


In [678]:
df = pd.concat(dat,axis=1)
ind=data.index
df.set_index(ind,inplace=True)
data=pd.concat([data,df],axis=1)
data.drop(categorical_col,axis=1,inplace=True)

In [679]:
data

,Area(total),Roof(Area),Lawn(Area),ANB,Grade,Price,Troom_0,Troom_1,Troom_2,Troom_3,...,API_31,API_32,API_33,API_34,API_35,API_36,API_37,API_38,API_39,API_40
id,,,,,,,,,,,,,,,,,,,,,
1,-0.984255,-1.076883,-1.181860,4,3,-0.423898,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.913637,-1.076883,-0.589151,1,1,0.849083,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-1.519558,1.136073,-1.012515,4,2,0.327403,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.135015,-1.076883,0.088230,1,3,-1.157071,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-1.470894,0.000000,0.596266,3,2,1.020291,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6996,-0.935591,0.000000,-1.012515,5,3,-0.643447,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6997,0.767645,1.204515,-1.520551,4,3,-1.404819,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6998,-0.935591,0.816678,-0.843169,4,3,-0.417856,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [680]:
target=data.pop('Grade')
targ=target.astype('category')

In [681]:
model = Sequential()
model.add(Dense(20, input_shape=(81,), activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(5, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [682]:
model.fit(data,targ,batch_size=20,epochs=50)

Epoch 1/60
7000/7000 [==============================] - 2s 306us/step - loss: 0.9483 - acc: 0.6151
Epoch 2/60
7000/7000 [==============================] - 1s 108us/step - loss: 0.3919 - acc: 0.8649
Epoch 3/60
7000/7000 [==============================] - 1s 106us/step - loss: 0.2629 - acc: 0.9171
Epoch 4/60
7000/7000 [==============================] - 1s 109us/step - loss: 0.2078 - acc: 0.9357
Epoch 5/60
7000/7000 [==============================] - 1s 114us/step - loss: 0.1739 - acc: 0.9454
Epoch 6/60
7000/7000 [==============================] - 1s 113us/step - loss: 0.1534 - acc: 0.9497
Epoch 7/60
7000/7000 [==============================] - 1s 108us/step - loss: 0.1390 - acc: 0.9536
Epoch 8/60
7000/7000 [==============================] - 1s 106us/step - loss: 0.1270 - acc: 0.9540
Epoch 9/60
7000/7000 [==============================] - 1s 104us/step - loss: 0.1196 - acc: 0.9559
Epoch 10/60
7000/7000 [==============================] - 1s 108us/step - loss: 0.1127 - acc: 0.9587
Epoch 11/

In [683]:
test=pd.read_csv('TEST.csv')

In [684]:
test.set_index('id',inplace=True)
test.head()

,Area(total),Troom,Nbedrooms,Nbwashrooms,Twashrooms,roof,Roof(Area),Lawn(Area),Nfloors,API,ANB,Price
id,,,,,,,,,,,,
1,348,9.0,5.0,3.0,5,NaN,NaN,84.0,5,91.0,6,4422$
2,311,6.0,5.0,5.0,6,NO,0.0,86.0,6,83.0,4,3432$
3,342,5.0,6.0,6.0,7,yes,120.0,85.0,4,95.0,5,4066$
4,329,7.0,7.0,4.0,6,NaN,NaN,105.0,5,97.0,6,4322$
5,307,6.0,2.0,3.0,4,no,0.0,76.0,2,64.0,4,3031$


In [685]:
text_lower(test,'roof')
clean_price(test,'Price')

In [686]:
for i in numerical_col:
    to_numeric(test,i)

for i in categorical_col:
    fill_categorical_na(test,i)
    
for j in numerical_col:
    fill_numerical_na(test,j)

9.0
4.0
3.0
5
yes
4
95.0
Area(total) 324.5304637769021
Roof(Area) 47.267195767195766
Lawn(Area) 90.13007883565797
Price 4025.7856926341315


In [687]:
for i in range(len(numerical_col)):
    scaller=num_scaller_ins[i]
    test[numerical_col[i]]=scaller.transform(test[numerical_col[i]].values.reshape(-1,1))

dat2=[]
for i in range(len(categorical_col)):
    values=np.array(test[categorical_col[i]].tolist())
    label_encoder = label_enco_ins[i]
    #label_encoder.fit(data[col])
    int_enco=label_encoder.transform(values)
    
    onehotencoder = one_hot_ins[i] 
    int_enco=int_enco.reshape(len(int_enco),1)
    onehot_encoded=onehotencoder.transform(int_enco)
    df5=pd.DataFrame(onehot_encoded).add_prefix(categorical_col[i]+'_')
    dat2.append(df5)
    

df2 = pd.concat(dat2,axis=1)
ind=test.index
df2.set_index(ind,inplace=True)
test=pd.concat([test,df2],axis=1)
test.drop(categorical_col,axis=1,inplace=True)    

In [688]:
test.head()

,Area(total),Roof(Area),Lawn(Area),ANB,Price,Troom_0,Troom_1,Troom_2,Troom_3,Troom_4,...,API_31,API_32,API_33,API_34,API_35,API_36,API_37,API_38,API_39,API_40
id,,,,,,,,,,,,,,,,,,,,,
1,1.108292,0.001470,-0.504479,6,0.776572,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.692272,-1.076883,-0.335133,4,-1.217497,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.816309,1.660795,-0.419806,5,0.059512,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.183678,0.001470,1.273647,6,0.575151,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,-0.886927,-1.076883,-1.181860,4,-2.025196,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [689]:
predicted=model.predict(test)

In [690]:
predicted

array([[3.25521441e-18, 1.00000000e+00, 7.56006457e-09, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 1.80297375e-20, 1.04954885e-02, 9.89504576e-01,
        3.73816089e-09],
       [3.02009180e-34, 2.69603534e-05, 9.99973059e-01, 4.16577249e-24,
        0.00000000e+00],
       ...,
       [0.00000000e+00, 1.65128015e-11, 1.00000000e+00, 2.35983803e-15,
        2.27785866e-35],
       [0.00000000e+00, 8.74594993e-20, 9.79266584e-01, 2.07334235e-02,
        4.46557422e-15],
       [0.00000000e+00, 3.42069995e-26, 2.29216275e-05, 9.99977112e-01,
        1.86605185e-08]], dtype=float32)

In [691]:
len(test)

3299

In [692]:
predicted_class=[]
for i in predicted:
    mx=i.max()
    print(mx)
    ind_mx = np.where(i == mx)
    print(ind_mx)
    predicted_class.append(ind_mx)

1.0
(array([1], dtype=int64),)
0.9895046
(array([3], dtype=int64),)
0.99997306
(array([2], dtype=int64),)
0.99318415
(array([1], dtype=int64),)
0.62537056
(array([4], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99999964
(array([1], dtype=int64),)
0.9999994
(array([2], dtype=int64),)
0.9998752
(array([3], dtype=int64),)
0.99997437
(array([3], dtype=int64),)
0.9999882
(array([3], dtype=int64),)
0.9999701
(array([2], dtype=int64),)
0.99997044
(array([3], dtype=int64),)
0.9999981
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.99976534
(array([1], dtype=int64),)
0.999747
(array([1], dtype=int64),)
0.99999976
(array([2], dtype=int64),)
0.9999982
(array([1], dtype=int64),)
0.9999292
(array([3], dtype=int64),)
0.99943477
(array([2], dtype=int64),)
0.9495427
(array([0], dtype=int64),)
0.9999329
(array([3], dtype=int64),)
0.9999244
(array([2], dtype=int64),)
0.97912073
(array([1], dtype=int64),)
1.0
(array([2], dtype=

(array([1], dtype=int64),)
0.9996019
(array([1], dtype=int64),)
0.99955744
(array([2], dtype=int64),)
0.99999905
(array([2], dtype=int64),)
0.628557
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9996476
(array([1], dtype=int64),)
0.9982667
(array([3], dtype=int64),)
0.9999964
(array([3], dtype=int64),)
0.99967027
(array([2], dtype=int64),)
0.99999917
(array([2], dtype=int64),)
0.99870753
(array([0], dtype=int64),)
0.9999014
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9988103
(array([3], dtype=int64),)
0.9901516
(array([3], dtype=int64),)
0.99988925
(array([1], dtype=int64),)
0.98916525
(array([2], dtype=int64),)
0.99936193
(array([3], dtype=int64),)
0.998946
(array([3], dtype=int64),)
0.9998379
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9999981
(array([1], dtype=int64),)
0.9993237
(array([2], dtype=int64),)
0.9983581
(array([3], dtype=int64),)
0.9999875
(array([1], dtype=int64),

1.0
(array([2], dtype=int64),)
0.7681189
(array([2], dtype=int64),)
0.99999666
(array([2], dtype=int64),)
0.99999976
(array([2], dtype=int64),)
0.99877375
(array([0], dtype=int64),)
0.53238523
(array([3], dtype=int64),)
0.999984
(array([2], dtype=int64),)
0.99999964
(array([1], dtype=int64),)
0.9929512
(array([3], dtype=int64),)
0.9999976
(array([1], dtype=int64),)
0.993796
(array([2], dtype=int64),)
0.97225845
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9999975
(array([2], dtype=int64),)
0.95041275
(array([2], dtype=int64),)
0.9100101
(array([3], dtype=int64),)
0.999985
(array([1], dtype=int64),)
0.9989089
(array([3], dtype=int64),)
0.597078
(array([3], dtype=int64),)
0.9432902
(array([1], dtype=int64),)
0.95331526
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9994746
(array([0], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9876643
(array([2], dtype=int64),)
0.99994457
(array([3], dtype=int64),)
1.0
(array([2], dtype=int6

(array([1], dtype=int64),)
0.99997115
(array([2], dtype=int64),)
0.9869666
(array([3], dtype=int64),)
0.99989605
(array([3], dtype=int64),)
0.99934834
(array([2], dtype=int64),)
0.99912924
(array([2], dtype=int64),)
0.9525416
(array([2], dtype=int64),)
0.999985
(array([3], dtype=int64),)
0.76565397
(array([3], dtype=int64),)
0.9999999
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9986689
(array([2], dtype=int64),)
0.99988246
(array([3], dtype=int64),)
0.99955374
(array([0], dtype=int64),)
0.9999994
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.997738
(array([3], dtype=int64),)
0.7578378
(array([3], dtype=int64),)
0.99623054
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99915934
(array([2], dtype=int64),)
0.9997235
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9603323
(array([1], dtype=int64),)
0.96575224
(array([2], dtype=int64),)
0.9996402
(array([3], dtype=int64),)
0.

(array([3], dtype=int64),)
0.99990773
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99999726
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99998164
(array([2], dtype=int64),)
0.9995977
(array([3], dtype=int64),)
0.99854374
(array([2], dtype=int64),)
0.99925834
(array([3], dtype=int64),)
0.9983991
(array([3], dtype=int64),)
0.9993019
(array([2], dtype=int64),)
0.9993587
(array([2], dtype=int64),)
0.99577004
(array([2], dtype=int64),)
0.88474613
(array([1], dtype=int64),)
0.99999726
(array([2], dtype=int64),)
0.9999094
(array([2], dtype=int64),)
0.9996176
(array([3], dtype=int64),)
0.9737604
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9998876
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9996898
(array([3], dtype=int64),)
0.9917795
(array([3], dtype=int64),)
0.99857724
(array([2], dtype=int64),)
0.9990276
(array([0], dtype=int64),)
0.99281466
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9999931
(array([3], dtype=int

(array([2], dtype=int64),)
0.9999788
(array([3], dtype=int64),)
0.99993634
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.99962544
(array([2], dtype=int64),)
0.9999999
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.99988794
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9999831
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9999546
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.619067
(array([3], dtype=int64),)
0.9996953
(array([3], dtype=int64),)
0.99784946
(array([3], dtype=int64),)
0.9997389
(array([2], dtype=int64),)
0.99993896
(array([3], dtype=int64),)
0.99787426
(array([2], dtype=int64),)
0.9999932
(array([2], dtype=int64),)
0.99966085
(array([0], dtype=int64),)
0.9422175
(array([1], dtype=int64),)
0.9902402
(array([3], dtype=int64),)
0.9999988
(array([2], dtype=int64),)
0.99945146
(array([0], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9997956
(arra

(array([3], dtype=int64),)
0.99982005
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99998057
(array([1], dtype=int64),)
0.9999999
(array([1], dtype=int64),)
0.99997497
(array([2], dtype=int64),)
0.73129773
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99827874
(array([3], dtype=int64),)
0.99797815
(array([3], dtype=int64),)
0.9954787
(array([2], dtype=int64),)
0.9820165
(array([2], dtype=int64),)
0.92515135
(array([0], dtype=int64),)
0.9997876
(array([3], dtype=int64),)
0.9998535
(array([0], dtype=int64),)
0.9999429
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9924697
(array([3], dtype=int64),)
0.99998975
(array([1], dtype=int64),)
0.9999056
(array([3], dtype=int64),)
0.99998486
(array([1], dtype=int64),)
0.99852175
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99992764
(array([3], dtype=int64),)
0.9999379
(array([3], dtype=int64),)
0.88850516
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),

(array([2], dtype=int64),)
0.97122556
(array([3], dtype=int64),)
0.99999917
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99995685
(array([1], dtype=int64),)
0.99993837
(array([3], dtype=int64),)
1.0
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9942339
(array([3], dtype=int64),)
0.99998426
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.98965377
(array([2], dtype=int64),)
0.999997
(array([1], dtype=int64),)
0.99818283
(array([3], dtype=int64),)
0.9998789
(array([1], dtype=int64),)
0.99953485
(array([1], dtype=int64),)
0.999966
(array([3], dtype=int64),)
0.77589613
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9881477
(array([3], dtype=int64),)
0.98329526
(array([3], dtype=int64),)
0.9992212
(array([3], dtype=int64),)
0.99531233
(array([2], dtype=int64),)
0.99981755
(array([2], dtype=int64),)
0.99582493
(array([3], dtype=int64),)
0.9998652
(array([1], dtype=int64),)
0.82930416
(array([3], dtype=int64),)
0.9999989
(array([2], dtype=i

(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9938332
(array([2], dtype=int64),)
0.9911641
(array([1], dtype=int64),)
0.9996513
(array([3], dtype=int64),)
0.99978465
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9503134
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99996495
(array([3], dtype=int64),)
0.9810369
(array([0], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.99999964
(array([2], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99246275
(array([3], dtype=int64),)
0.99958783
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
1.0
(array([1], dtype=int64),)
0.9993585
(array([3], dtype=int64),)
0.9996184
(array([0], dtype=int64),)
0.99856746
(array([3], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.99991333
(array([3], dtype=int64),)
0.9999999
(array([2], dtype=int64),)
0.9999975
(array([1], dtype=int64),)
1.0
(array([2], dtype=int64),)
0.9967167
(array([2], dtype=int64

In [693]:
predicted[0][0]

3.2552144e-18

In [694]:
predicted_grade=[]
for i in predicted_class:
    inv=grade_encoder.inverse_transform(i)
    predicted_grade.append(inv[0])

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [695]:
# from sklearn.metrics import classification_report
# print(classification_report(tgg,predicted_grade))

In [696]:
result=pd.DataFrame()

In [697]:
result['id']=test.index

In [698]:
result['Grade']=predicted_grade

In [699]:
result

,id,Grade
0,1,B
1,2,D
2,3,C
3,4,B
4,5,E
...,...,...
3294,3295,C
3295,3296,D
3296,3297,C
3297,3298,C


In [700]:
result.to_csv('result.csv',index=False)

In [701]:
sample=pd.read_csv('SAMPLE.csv')
sample.head()

,id,Grade
0,1,C
1,2,B
2,3,E
3,4,E
4,5,B


In [702]:
result3=pd.read_csv('result.csv')
result3.head()

,id,Grade
0,1,B
1,2,D
2,3,C
3,4,B
4,5,E


In [703]:
len(result3)

3299

In [704]:
Step 1:  (Data Preprocessing)
I have replaced the Null values of numerical variable with the mean value and the Null value of Categorical variable with the most common value.
Then I did some cleaning process of various columns. for example ,removed the $ sign from the price to make it as numeric value. 
Then, I applied Label encoder and One Hot Encoder to encode the categorical Variables and created dummy variable for each class or category.
 Applied StandardScaler to scale the numerical variables to become in range of -1 to 1 which will be easier for our model to learn and get better performance.

Step 2: Model Building and Training:
I have used the keras sequential model with  2 hidden layer and one output layer.
The hidden layers have  'relu' as activation function and the last output layer has 'softmax' as activation function  as our target column has multi label categories. 
Loss function used  is 'sparse_categorical_crossentropy' as it is a multi classifier model.

SyntaxError: invalid syntax (<ipython-input-704-00859a24421d>, line 1)